In [7]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

#definimos las credenciales

client_id = 'b6841f7c97974c8aacbdf448dae8677f'
client_secret = 'bc15cade85bc41f88d451fbb2c832eff'
redirect_url = 'http://localhost:8888/callback/'

#scope (permisos) para acceder a tus playlist
scope = 'playlist-read-private'

#autenticacion 
sp = spotipy.Spotify(auth_manager = SpotifyOAuth(
    client_id = client_id,
    client_secret = client_secret,
    redirect_uri = redirect_url,
    scope = scope
))

# Obtener tus playlists
playlists = sp.current_user_playlists()
playlists
#mostrar lso nombres de las playlist y sus IDS
for playlist in playlists['items']:
    print(f"Playlist: {playlist['name']}, ID: {playlist['id']}")

Playlist: My recommendation playlist, ID: 5bQpZDbY0JqW2wvmC5SpB7
Playlist: DJ, ID: 37i9dQZF1EYkqdzj48dyYq
Playlist: duchazo, ID: 4eOY4P1EyXudJoSmZu4uIu
Playlist: myhead, ID: 4Q3DMQ58r0PepiqMMhhGqW


In [8]:
#especifico el ID de mi playist a explorar
playlist_id = '4Q3DMQ58r0PepiqMMhhGqW'

#obtengo los resultados de las canciones de mi playlist
results = sp.playlist_tracks(playlist_id)

In [9]:
# Lista para almacenar todas las canciones
all_tracks = []

# Iterar mientras haya canciones en la respuesta
while results['items']:
    for item in results['items']:
        track = item['track']
        artists_id = track['artists'][0]['id']
        artists_info = sp.artist(artists_id)
        genres = artists_info['genres']
        album = track['album']
        artist_id = track['artists'][0]['id']  # ID del artista principal
        album_image_url = album['images'][0]['url'] if album['images'] else None
        
        # Obtener la información del artista, incluyendo la imagen de perfil
        artist_info = sp.artist(artist_id)
        artist_image_url = artist_info['images'][0]['url'] if artist_info['images'] else None


        all_tracks.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'duration_sec': track['duration_ms'] / 1000,
            'popularity': track['popularity'],
            'release_date': track['album']['release_date'],
            'album_image_url': album_image_url,
            'artist_image_url': artist_image_url,
            'genres': ', '.join(genres),
            'track_id':track['id']
        })
    
    # Incrementar el offset para obtener el siguiente bloque de canciones
    offset = len(all_tracks)
    results = sp.playlist_tracks(playlist_id, offset=offset)

# Crear DataFrame con todos los datos de las canciones
playlist_df = pd.DataFrame(all_tracks)


In [10]:
# Separar la columna 'genres' por comas y expandir los resultados en columnas
genres_split = playlist_df['genres'].str.split(', ', expand=True)

# Renombrar las columnas resultantes
genres_split.columns = [f'genre_{i+1}' for i in range(genres_split.shape[1])]

# Convertir todos los valores de la columna 'genres' a cadenas por si acaso hay valores no string
playlist_df['genres'] = playlist_df['genres'].astype(str)

# Contar la cantidad de géneros en cada fila
playlist_df['genre_count'] = playlist_df['genres'].apply(lambda x: len(x.split(', ')))

# Unir el DataFrame original con el DataFrame de géneros separados
playlist_df = pd.concat([playlist_df, genres_split], axis=1)

In [11]:
playlist_df.to_csv('my_playlist.csv')

In [12]:
playlist_df

,name,artist,album,duration_sec,popularity,release_date,album_image_url,artist_image_url,genres,track_id,genre_count,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7
0,Like That,Future,WE DON'T TRUST YOU,267.706,85,2024-03-22,https://i.scdn.co/image/ab67616d0000b273a46b07...,https://i.scdn.co/image/ab6761610000e5eb7565b3...,"atl hip hop, hip hop, rap, southern hip hop, trap",2tudvzsrR56uom6smgOcSf,5,atl hip hop,hip hop,rap,southern hip hop,trap,None,None
1,Feel The Fiyaaaah (with A$AP Rocky & feat. Tak...,Metro Boomin,HEROES & VILLAINS,189.426,67,2022-12-02,https://i.scdn.co/image/ab67616d0000b273c4fee5...,https://i.scdn.co/image/ab6761610000e5ebdf9a15...,rap,3SsJ17EnPIu1B4GZshqjIS,1,rap,None,None,None,None,None,None
2,Count Me Out,Kendrick Lamar,Mr. Morale & The Big Steppers,283.642,75,2022-05-13,https://i.scdn.co/image/ab67616d0000b2732e0211...,https://i.scdn.co/image/ab6761610000e5eb437b9e...,"conscious hip hop, hip hop, rap, west coast rap",6BU1RZexmvJcBjgagVVt3M,4,conscious hip hop,hip hop,rap,west coast rap,None,None,None
3,CARNIVAL,¥$,VULTURES 1,264.324,55,2024-02-09,https://i.scdn.co/image/ab67616d0000b2730a31b4...,https://i.scdn.co/image/ab6761610000e5ebf4e110...,,3w0w2T288dec0mgeZZqoNN,1,,None,None,None,None,None,None
4,Praise God,Kanye West,Donda,226.652,72,2021-08-29,https://i.scdn.co/image/ab67616d0000b273cad190...,https://i.scdn.co/image/ab6761610000e5eb6e835a...,"chicago rap, hip hop, rap",0WSEq9Ko4kFPt8yo3ICd6T,3,chicago rap,hip hop,rap,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,Wet Dreamz,J. Cole,2014 Forest Hills Drive,239.266,80,2014-12-09,https://i.scdn.co/image/ab67616d0000b273c6e094...,https://i.scdn.co/image/ab6761610000e5eb4b053c...,"conscious hip hop, hip hop, north carolina hip...",4tqcoej1zPvwePZCzuAjJd,4,conscious hip hop,hip hop,north carolina hip hop,rap,None,None,None
183,LUST.,Kendrick Lamar,DAMN.,307.880,64,2017-04-14,https://i.scdn.co/image/ab67616d0000b2738b52c6...,https://i.scdn.co/image/ab6761610000e5eb437b9e...,"conscious hip hop, hip hop, rap, west coast rap",06FCvd7rrRcF3DdvWH5Isp,4,conscious hip hop,hip hop,rap,west coast rap,None,None,None
184,YAH.,Kendrick Lamar,DAMN.,160.293,66,2017-04-14,https://i.scdn.co/image/ab67616d0000b2738b52c6...,https://i.scdn.co/image/ab6761610000e5eb437b9e...,"conscious hip hop, hip hop, rap, west coast rap",4BHSjbYylfOH5WAGusDyni,4,conscious hip hop,hip hop,rap,west coast rap,None,None,None
185,Bank Account,21 Savage,Issa Album,220.306,73,2017-07-06,https://i.scdn.co/image/ab67616d0000b273271934...,https://i.scdn.co/image/ab6761610000e5eb4f8f76...,"atl hip hop, hip hop, rap",5eqK0tbzUPo2SoeZsov04s,3,atl hip hop,hip hop,rap,None,None,None,None
